In [ ]:
from scipy.fft import fft, ifft
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import animation
from scipy.integrate import odeint, solve_ivp
plt.rcParams['text.usetex'] = True
import warnings
from IPython.display import HTML
import time

start = time.time()
def solitongenerator(L,v,g,starttime,endtime,initialposition,amplitude,wavelength):


    L   = L
    nx  = 100
    a   = .1
    ntflow = .1
    g= g
    lamb = 1
    tol = 1e-10
    tend = ntflow*L/a
    v = v
    
    
    # In[71]:
    
    
    #---------- solution domain, initial condition
    
    dx = L/nx     # not L/(nx-1)
    x = np.linspace(0.0, L-dx, nx)
    u0 = np.concatenate([np.ones(len(x))*x,np.ones(len(x))*v]) #[val for pair in zip(np.ones(len(x)), np.ones(len(x))*v) for val in pair]
    
    
    # In[72]:
    
    
    np.cosh(0.1)
    
    
    # In[73]:
    
    
    #---------- solve the problem and plot
    
    def rhsf(t, u):
        N = nx
        n = np.arange(N); n[int(N/2)+1:]-= N
        i = 1j    
        def xy(k):
            #print(ifft((2*np.pi*i*n/L)**1*fft(k)).real)
            if t==0:
                return np.ones(len(k))
            ret = ifft((2*np.pi*i*n/L)**1*fft(k)).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def xyy(k):
            ret= ifft((2*np.pi*i*n/L)**1*fft(xy(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def xyyy(k):
            ret = ifft((2*np.pi*i*n/L)**2*fft(xy(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def xyyyy(k):
            ret = ifft((2*np.pi*i*n/L)**3*fft(xy(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def R0(k):
            vals = np.array([])
            for i in k:
                try:
                    vals=np.append(vals,amplitude * 1/(np.cosh(wavelength*(i-initialposition))))
                except RuntimeWarning as e:
                    #print(f'Error in R0 here is u')
                    #print(k)
                    #print(e, 'set to 0')
                    vals=np.append(vals,0.0001)
            #print(vals)
            ret = vals
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def rho0(k):
            ret = R0(k) **2
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def gamma0(k):
            #return rho0(k)
            ret = rho0(k) *xy(k)
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def P0(k):
            try:
                np.abs(R0(k))
                #np.sqrt(gamma0(k))
            except RuntimeWarning as e :
                print(f'Error in P0 here is u')
                print(k)
                print(e)
                pass
            ret = np.abs(R0(k))
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def P0y(k):
            ret = ifft((2*np.pi*i*n/L)**1*fft(P0(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def P0yy(k):
            ret = ifft((2*np.pi*i*n/L)**2*fft(P0(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def Q0(k):
            try:
                -0.5 *P0yy(k) / P0(k)
            except RuntimeWarning as e:
                print(f'Error in Q0 here is u')
                print(k)
                print(e)
            ret = -0.5 *P0yy(k) / P0(k)
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        def Q0y(k):
            ret = ifft((2*np.pi*i*n/L)**1*fft(Q0(k))).real
            idx = np.argwhere(np.abs(ret) < 1e-50)
            ret[idx]=tol
            return ret
        
        
        try:
            #results = -Q0y(u[:nx]) -2*g*P0(u[:nx])*P0y(u[:nx])
            try:
                term1 = -Q0y(u[:nx])/(xy(u[:nx])**3) #*P0(u[:nx])**2
            except:
                term1 = []
                for it in range(len(x)):
                    try:
                        term1.append(-Q0y(u[:nx])[it]/(xy(u[:nx])[it]**3))
                    except:
                        term1.append(0)
            
            
            try:
                term2 = (4*Q0(u[:nx]) + (P0y(u[:nx])/P0(u[:nx]))**2 )*xyy(u[:nx])/(xy(u[:nx]) **4)
            except:
                term2 = []
                for it in range(len(x)):
                    try:
                        term2.append((4*Q0(u[:nx])[it] + (P0y(u[:nx])[it]/P0(u[:nx])[it])**2 )*xyy(u[:nx])[it]/(xy(u[:nx])[it] **4))
                    except:
                        term2.append(0)
                
                
                
            try:
                term3 = 3* (P0y(u[:nx])/P0(u[:nx])) * (xyy(u[:nx])**2 /(xy(u[:nx])**5))#*P0(u[:nx])**2
            except:
                term3 = []
                for it in range(len(x)):
                    try:
                        term3.append(3* (P0y(u[:nx])[it]/P0(u[:nx])[it]) * (xyy(u[:nx])[it]**2 /(xy(u[:nx])[it]**5)))
                    except:
                        term3.append(0)
            
            
            
            try:
                term4 = - 1* (P0y(u[:nx])/P0(u[:nx])) * (xyyy(u[:nx])**1 /(xy(u[:nx])**4))#*P0(u[:nx])**2
            except:
                term4 = []
                for it in range(len(x)):
                    try:
                        term4.append(- 1* (P0y(u[:nx])[it]/P0(u[:nx])[it]) * (xyyy(u[:nx])[it]**1 /(xy(u[:nx])[it]**4)))
                    except:
                        term4.append(0)
                
                
                
            try:
                term5 = -(5/2)*((xyy(u[:nx])**3)/(xy(u[:nx])**6))#*P0(u[:nx])**2
            except:
                term5 = []
                for it in range(len(x)):
                    try:
                        term5.append(-(5/2)*((xyy(u[:nx])[it]**3)/(xy(u[:nx])[it]**6)))
                    except:
                        term5.append(0)
                
                
                
                
            try:
                term6 = 2 * xyy(u[:nx])*xyyy(u[:nx])/(xy(u[:nx])**5)##*P0(u[:nx])**2
            except:
                term6 = []
                for it in range(len(x)):
                    try:
                        term6.append(2 * xyy(u[:nx])[it]*xyyy(u[:nx])[it]/(xy(u[:nx])[it]**5))
                    except:
                        term6.append(0)
                
                
                
                
            try:
                term7 = - 1/4 *xyyyy(u[:nx])/xy(u[:nx])**4#*P0(u[:nx])**2
            except:
                term7 = []
                for it in range(len(x)):
                    try:
                        term7.append(- 1/4 *xyyyy(u[:nx])[it]/xy(u[:nx])[it]**4)
                    except:
                        term7.append(0)
                
                
                
                
            try:
                term8 = - 2*g*P0(u[:nx])*P0y(u[:nx])/xy(u[:nx])**2#*P0(u[:nx])**2
            except:
                term8 = []
                for it in range(len(x)):
                    try:
                        term8.append(- 2*g*P0(u[:nx])[it]*P0y(u[:nx])[it]/xy(u[:nx])[it]**2)
                    except:
                        term8.append(0)
                
                
                
                
            try:
                term9 = g*P0(u[:nx])**2 *xyy(u[:nx])/xy(u[:nx])**3#*P0(u[:nx])**2
            except:
                term9 = []
                for it in range(len(x)):
                    try:
                        term9.append(g*P0(u[:nx])[it]**2 *xyy(u[:nx])[it]/xy(u[:nx])[it]**3)
                    except:
                        term9.append(0)
                
                
                
                
            results = term1 + term2 + term3 + term4 +term5 + term6 +term7 +term8 + term9 
            #print('xy',xy(u[:nx]))
            #print('xy6',xy(u[:nx])**6)
            #print('p0',P0(u[:nx]))
            #print('p02',P0(u[:nx])**2)
            #results = results/xy(u[:nx])**6 /P0(u[:nx])**2
            #print(u[:nx])
            #print(xy(u[:nx]))
        except RuntimeWarning as e:
            print(f'Error in results here is u')
            print(u)
            print(e)
            try:
                term1 = -Q0y(u[:nx])/(xy(u[:nx])**3) #*P0(u[:nx])**2
            except:
                term1 = 0
            try:
                term2 = (4*Q0(u[:nx]) + (P0y(u[:nx])/P0(u[:nx]))**2 )*xyy(u[:nx])/(xy(u[:nx]) **4)
            except:
                term2 = 0
            try:
                term3 = 3* (P0y(u[:nx])/P0(u[:nx])) * (xyy(u[:nx])**2 /(xy(u[:nx])**5))#*P0(u[:nx])**2
            except:
                term3 = 0
            
            try:
                term4 = - 1* (P0y(u[:nx])/P0(u[:nx])) * (xyyy(u[:nx])**1 /(xy(u[:nx])**4))#*P0(u[:nx])**2
            except:
                term4 = 0
            try:
                term5 = -(5/2)*((xyy(u[:nx])**3)/(xy(u[:nx])**6))#*P0(u[:nx])**2
            except:
                term5 = 0
            try:
                term6 = 2 * xyy(u[:nx])*xyyy(u[:nx])/(xy(u[:nx])**5)##*P0(u[:nx])**2
            except:
                term6 = 0
            try:
                term7 = - 1/4 *xyyyy(u[:nx])/xy(u[:nx])**4#*P0(u[:nx])**2
            except:
                term7 = 0
            try:
                term8 = - 2*g*P0(u[:nx])*P0y(u[:nx])/xy(u[:nx])**2#*P0(u[:nx])**2
            except:
                term8 = 0
            try:
                term9 = g*P0(u[:nx])**2 *xyy(u[:nx])/xy(u[:nx])**3#*P0(u[:nx])**2
            except:
                term9 = 0
            results = term1 + term2 + term3 + term4 +term5 + term6 +term7 +term8 + term9 
            #print('xy',xy(u[:nx]))
            #print('xy6',xy(u[:nx])**6)
            #print('p0',P0(u[:nx]))
            #print('p02',P0(u[:nx])**2)
            #results = results/xy(u[:nx])**6 /P0(u[:nx])**2
            #print(u[:nx])
            #print(xy(u[:nx]))
    
        #print(xyy(u[nx:]))
        #print(u[0::2])
        print(t)
        return np.concatenate([u[nx:] , results]) #[val for pair in zip(u[1::2], results) for val in pair]
    
    
    # In[ ]:
    
    
    
    
    
    # In[74]:
    
    
    start = starttime
    end = endtime
    tspan = np.linspace(start,end,1000)
    dt = tspan[1]-tspan[0]
    class maximise():
        def __init__(self):
            pass
        def f(self,tol):
            y= solve_ivp(rhsf,(start,end),u0,method = 'BDF',atol=tol,t_eval=tspan)
            if y.status == -1:
                return [y,-1]
            if y.status == 0:
                return [y,1]
        def bisection(self,a,b,N):
            if self.f(a)[1]*self.f(b)[1] >= 0:
                print("Bisection method fails.")
                self.tol=b
                self.f_b_n=self.f(b)
                return b
                return None
            a_n = a
            b_n = b
            for n in range(1,N+1):
                m_n = (a_n + b_n)/2
                self.f_m_n = self.f(m_n)
                self.f_a_n=self.f(a_n)
                self.f_b_n=self.f(b_n)
                if self.f_a_n[1]*self.f_m_n[1] < 0:
                    a_n = a_n
                    b_n = m_n
                
                elif self.f_b_n[1]*self.f_m_n[1] < 0:
                    a_n = m_n
                    b_n = b_n
                elif self.f_m_n[1] == 0:
                    print("Found exact solution.")
                    return m_n
                else:
                    print("Bisection method fails.")
                    self.tol=b_n
                    return b_n
            self.tol=b_n
            return b_n
    
    
    
    # In[75]:
    
    
    y=maximise()
    y.bisection(1e5,1e20,1)
    print(y.f_b_n)
    print(y.tol)
    
    
    # In[76]:
    
    
    y=y.f_b_n[0]
    
    
    # In[101]:
    
    
    fig,ax=plt.subplots()
    for i in y.y[0:nx]:
        ax.plot(y.t,i, '-', lw=2,label=f'$y={round(i[0],2)}$') if i[0]<2.5 else ax.plot(y.t,i, '-', lw=2)
    #ax.plot(y.t,y.y[0], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[1], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[2], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[3], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[4], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[1], '-', color='red', lw=2)
    plt.title(f'Graph of all solutions $L={L}, nx={nx}$')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('$t$')
    plt.ylabel(r'$x(y,t)$');





    fig,ax=plt.subplots()
    for j in y.y[0:nx]:
        xdot = []
        for i in range(len(j) - 1):
            xdot.append((j[i+1] - j[i])/dt)
        if np.var(xdot) < 1e-1:
            ax.plot(y.t,j, '-', lw=2,label=f'$y={round(j[0],2)}$') if j[0] < 2.5 else ax.plot(y.t,j, '-', lw=2)
    #ax.plot(y.t,y.y[0], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[1], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[2], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[3], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[4], '-', color='red', lw=2)
    #ax.plot(y.t,y.y[1], '-', color='red', lw=2)
    plt.title(f'Graph of filtered solutions $L={L}, nx={nx}$')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('$t$')
    plt.ylabel(r'$x(y,t)$');
    
    
    # In[78]:
    
    
    fig,ax=plt.subplots()
    tspan2 = (tspan+ dt/2)[0:-1]
    for j in y.y[0:nx]:
        xdot = []
        for i in range(len(j) - 1):
            xdot.append((j[i+1] - j[i])/dt)
        ax.plot(tspan2,xdot, '-', lw=2,label=f'$y={round(j[0],2)}$') if j[0]<2.5 else ax.plot(tspan2,xdot, '-', lw=2)
    plt.title(f'$L={L}, nx={nx}$')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('$t$')
    plt.ylabel(r'$\dot{x}(y,t)$');
    
    
    # fig,ax=plt.subplots()
    # tspan2 = (tspan+ dt/2)[0:-1]
    # for j in y.y[round(nx/2):nx]:
    #     xdot = []
    #     for i in range(len(j) - 1):
    #         xdot.append((j[i+1] - j[i])/dt)
    #     ax.plot(tspan2,xdot, '-', lw=2,label=f'{round(j[0],2)}')
    # plt.title(f'xdot L={L}, nx={nx}')
    # plt.rc('font', size=14)
    # ax.legend(bbox_to_anchor=(1.1, 1.05))
    # #plt.gca().set_xlim([0,0.072])
    # #plt.gca().set_ylim([-10,10])
    # plt.xlabel('t')
    # plt.ylabel(r'xdot(y,t)');
    
    
    # In[26]:
    
    
    fig,ax=plt.subplots()
    tspan2 = (tspan+ dt/2)[0:-1]
    tspan3 = (tspan2 + tspan2[1]/2 - tspan2[0]/2)[0:-1]
    for j in y.y[0:nx]:
        xdot = []
        xddot = []
        for i in range(len(j) - 1):
            xdot.append((j[i+1] - j[i])/dt)
        for i in range(len(xdot)-1):
            xddot.append((xdot[i+1]-xdot[i])/(tspan2[1]/2 - tspan2[0]/2))
        ax.plot(tspan3,xddot, '-', lw=2,label=f'$y={round(j[0],2)}$') if j[0]<2.5 else ax.plot(tspan3,xddot, '-', lw=2,)
    plt.title(f'$L={L}, nx={nx}$')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('$t$')
    plt.ylabel(r'$\ddot{x}(y,t)$');
    
    
    # fig,ax=plt.subplots()
    # tspan2 = (tspan+ dt/2)[0:-1]
    # tspan3 = (tspan2 + tspan2[1]/2 - tspan2[0]/2)[0:-1]
    # for j in y.y[round(nx/2):nx]:
    #     xdot = []
    #     xddot = []
    #     for i in range(len(j) - 1):
    #         xdot.append((j[i+1] - j[i])/dt)
    #     for i in range(len(xdot)-1):
    #         xddot.append((xdot[i+1]-xdot[i])/(tspan2[1]/2 - tspan2[0]/2))
    #     ax.plot(tspan3,xddot, '-', lw=2,label=f'{round(j[0],2)}')
    # plt.title(f'xddot L={L}, nx={nx}')
    # plt.rc('font', size=14)
    # ax.legend(bbox_to_anchor=(1.1, 1.05))
    # #plt.gca().set_xlim([0,0.072])
    # #plt.gca().set_ylim([-10,10])
    # plt.xlabel('t')
    # plt.ylabel(r'xddot(y,t)');
    
    
    
    # In[ ]:
    
    
    
    
    
    # In[94]:
    
    
    
    
    
    # In[104]:
    
    
    solitonic = [i for i in y.y[0:nx] if np.abs(i[1]/dt - i[0]/dt  - i[-1]/dt +i[-2]/dt) < 1e-2]
    
    
    # In[114]:
    
    
    solitonicinv = [list(x) for x  in zip(*solitonic)]
    
    
    # In[132]:
    
    
    len(tspan)
    
    
    # In[157]:
    
    
    from statistics import mean
    def R(x,t):
        return (amplitude * 1/(np.cosh(wavelength*(x-v*t-initialposition))))**2
    
    
    # In[158]:
    
    
    fig,ax=plt.subplots()
    numberoftimeplots = 10
    for i in range(0,len(tspan),round(len(tspan)/numberoftimeplots)):
        ax.plot(solitonicinv[i],R(solitonicinv[i],y.t[i]), '-', lw=2,label=f'$t={round(y.t[i])}$')
    plt.title(f'$L={L}, nx={nx}$')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('$x$')
    plt.ylabel('$\\rho$');
    plt.savefig
    return [solitonicinv,[R(solitonicinv[i],y.t[i]) for i in range(0,len(tspan))]]


In [ ]:

soliton1 = solitongenerator(20,1,-4,0,10,4,0.5,1)
print('FIrst done')

end = time.time()
print(end - start)

In [ ]:
soliton2 = solitongenerator(20,-1,-4,0,10,10,0.5,1)

In [ ]:
soliton12 = [soliton1,soliton2]
soliton1 = soliton12[0]
soliton2 = soliton12[1]

In [ ]:
newx=[]
newrho=[]
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs
for t in range(1000):
    for idx, val in enumerate(soliton2[0][t]):
        for val2 in soliton1[0][t]:
            if np.abs(val2-val) <0.2:
                soliton2[0][t][idx] = val2
    xtemp = list(soliton1[0][t]) + list(soliton2[0][t])
    rhotemp = list(soliton1[1][t]) + list(soliton2[1][t])
    
    rhotemp = [rho for _, rho in sorted(zip(xtemp, rhotemp))]
    xtemp.sort()

    for i in xtemp:
        dupes = list_duplicates_of(xtemp, i)
        if len(dupes)>2:
            for j in dupes[1:]:
                rhotemp[dupes[0]] += rhotemp[j]
            for count,j in enumerate(dupes[1:]):
                xtemp.pop(j-count)
                rhotemp.pop(j-count)
        if len(dupes)==2:
            rhotemp[dupes[0]] += rhotemp[dupes[1]]
            xtemp.pop(dupes[1])
            rhotemp.pop(dupes[1])
    newx.append(xtemp)
    newrho.append(rhotemp)

In [ ]:
'''for t in range(1000):
    overlap1 = [i for i, j in zip(soliton1[0][t],soliton2[0][t]) if np.abs(i-j)<0.5e0]
    print(overlap1)
    overlap2 = [j for i, j in zip(soliton1[0][t],soliton2[0][t]) if np.abs(i-j)<0.5e0]
    tempx = set(soliton1[0][t]) | set(soliton2[0][t])
    print(tempx)
    tempx = list(set(tempx) - set(set(tempx) - set(overlap2)))
    print(tempx)
    tempx.sort()
    newx.append(list(tempx))
    rho =[]
    for x in list(tempx):
        if x in overlap1:
            idx1 = soliton1[0][t].index(x)
            tempidx = overlap1.index(x)
            y = overlap2[tempidx]
            idx2 = soliton2[0][t].index(y)
            rho.append(soliton1[1][t][idx1] + soliton2[1][t][idx2])
        elif x in soliton1[0][t]:
            idx1 = soliton1[0][t].index(x)
            rho.append(soliton1[1][t][idx1])
        elif x in soliton2[0][t]:
            idx1 = soliton2[0][t].index(x)
            rho.append(soliton2[1][t][idx1])
    newrho.append(rho)'''

In [ ]:
len(soliton1[1][0])

In [ ]:
len(newx)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8,4))

line1, = ax1.plot([],[],lw=3)
line2, = ax1.plot([],[],lw=3)
ax1.set_xlim((0,10)); ax1.set_ylim((0.,1))
plt.close()

def plot_frame(i):
    line1.set_data(soliton1[0][0::2][i],soliton1[1][0::2][i]);
    line2.set_data(soliton2[0][0::2][i],soliton2[1][0::2][i]);
    ax1.set_title('t=')

# Animate the solution
anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                   frames=len(soliton1[0][0::2]),
                                   interval=5)

HTML(anim.to_jshtml())

In [ ]:
    fig,ax=plt.subplots()
    numberoftimeplots = 10

    ax.plot(newx[5],newrho[5], '-', lw=2)
    plt.title(f'xdot ')
    plt.rc('font', size=14)
    ax.legend(bbox_to_anchor=(1.1, 1.05))
    #plt.gca().set_xlim([0,0.072])
    #plt.gca().set_ylim([-10,10])
    plt.xlabel('x')
    plt.ylabel(r'rho');

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8,4))

line1, = ax1.plot([],[],lw=3)
line2, = ax1.plot([],[],lw=3)
line3, = ax1.plot([],[],lw=3)
ax1.set_xlim((0,12.5)); ax1.set_ylim((0.,1))
plt.close()

def plot_frame(i):
    line1.set_data(newx[0::2][i],newrho[0::2][i]);
    line2.set_data(soliton1[0][0::2][i],soliton1[1][0::2][i]);
    line3.set_data(soliton2[0][0::2][i],soliton2[1][0::2][i]);
    ax1.set_title('t=')

# Animate the solution
anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                   frames=500,
                                   interval=5)

HTML(anim.to_jshtml())

#writervideo = matplotlib.animation.FFMpegWriter(fps=60) 
#anim.save('increasingStraightLine.gif', writer=writervideo) 


In [ ]:
#writervideo = matplotlib.animation.FFMpegWriter(fps=60) 
#anim.save('increasingStraightLine.mp4', writer=writervideo) 

#writergif = matplotlib.animation.PillowWriter(fps=15) 
#anim.save('increasingStraightLine.gif', writer=writergif)

In [ ]:
fig,ax=plt.subplots()
numberoftimeplots = 10
for i in range(0,1000,200):
    ax.plot(newx[i],newrho[i], '-', lw=2)
plt.title(f'xdot ')
plt.rc('font', size=14)
ax.legend(bbox_to_anchor=(1.1, 1.05))
#plt.gca().set_xlim([0,0.072])
#plt.gca().set_ylim([-10,10])
plt.xlabel('t')
plt.ylabel(r'u(t)');

In [ ]:
end = time.time()
print(end - start)

In [ ]:
import PIL